일별로 기록된 센서데이터의 모션 센서의 감지기록을 바탕으로 시계열 분석을 실시한다. 
해당 분석을 통해 시간별 구획의 유동인구 파악과 사용이 많은 조명과 센서를 파악한다.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pymysql
import pandas as pd
import plotly as px
import math
import os
#MYSQL 연결하기
report_db =pymysql.connect(host='xxx.xxx.xx.xx',user='root',password='xxxxx',db='report_old_data',charset='utf8',autocommit=True,cursorclass=pymysql.cursors.DictCursor)

cursor = report_db.cursor(pymysql.cursors.DictCursor)
sql = "SELECT * FROM bssm_nt_led_status_hist_20221201_20221231;"
cursor.execute(sql)
result = cursor.fetchall()
#result1 = result.query('"2022-05-01 00:00:00"<= hist_dt <= "2022-06-01 00:00:00"')

result = pd.DataFrame(result)
result
###################################################################################
data_main = result
data1 = data_main

data1


,gid,did,mid,hist_dt,building_id,onoff,level,colortemp,hue,saturation,brightness,x,y,r,g,b,report_uspace_yn,operator
0,1,41,0,2022-12-01 00:02:43.439,101,1,0,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,None
1,1,41,0,2022-12-01 00:07:43.439,101,1,0,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,None
2,1,41,0,2022-12-01 00:12:43.438,101,1,0,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,None
3,1,41,0,2022-12-01 00:17:43.438,101,1,0,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,None
4,1,41,0,2022-12-01 00:20:20.785,101,1,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,scenario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999393,57,1199,0,2022-12-29 17:03:45.416,101,0,100,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,ai
19999394,57,1200,0,2022-12-29 17:03:45.442,101,0,100,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,ai
19999395,57,1201,0,2022-12-29 17:03:45.453,101,0,100,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,ai
19999396,57,1202,0,2022-12-29 17:03:45.453,101,0,100,5700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,ai


In [2]:
fault12 = pd.read_csv('nt_device_status_hist_202212.csv')
fault12x = fault12.query('"2022-12-05 00:00:00"<= hist_dt <= "2022-12-11 23:59:59"')
fault12x = fault12x.drop('gid',axis=1)
fault12x = fault12x.drop('building_id',axis=1)
fault12x = fault12x.drop('old_comm_status_code',axis=1)
fault12x = fault12x.drop('old_fault_status',axis=1)
fault12x = fault12x.drop('operator',axis=1)
fault12x = fault12x.sort_values(by=['hist_dt'], axis = 0, ascending = True)
fault12x = fault12x.reset_index(drop=True)

/home/aloha/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning:

Columns (4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.



In [3]:
fault12x

,did,hist_dt,device_fault,module_cnt,report_uspace_yn
0,124,2022-12-05 00:00:02.386000,\N,3,\N
1,125,2022-12-05 00:00:02.386000,\N,3,\N
2,126,2022-12-05 00:00:02.386000,\N,3,\N
3,127,2022-12-05 00:00:02.386000,\N,3,\N
4,53,2022-12-05 00:00:04.333000,\N,3,\N
...,...,...,...,...,...
5745144,65,2022-12-11 23:59:57.766000,\N,3,\N
5745145,64,2022-12-11 23:59:57.766000,\N,3,\N
5745146,83,2022-12-11 23:59:57.766000,\N,3,\N
5745147,63,2022-12-11 23:59:57.766000,\N,3,\N


In [92]:
df1_report1 = fault12x


gw154 = df1_report1.query('did == 1|did == 2|did == 4|did == 5|did == 7|did == 159|did == 15|did == 16|did == 17|did == 162 \
                          |did == 18|did == 19|did == 25|did == 26|did == 27|did == 28|did == 29|did == 30|did == 36|did == 37')

gw155 = df1_report1.query('did == 160|did == 8|did == 9|did == 10|did == 11|did == 13|did == 14|did == 161|did == 20|did == 21| \
                           did == 22|did == 23|did == 24|did == 31|did == 32|did == 33|did == 34|did == 35|did == 38|did == 39 ')

gw156a = df1_report1.query('did == 50|did == 51|did == 57|did == 58|did == 67|did == 68|did == 77|did == 78|did == 89|did == 90| \
                            did == 91|did == 92|did == 140|did == 141 ')

gw156b = df1_report1.query('did == 93')

gw156c = df1_report1.query('did == 52|did == 53|did == 69|did == 70|did == 79|did == 80|did == 87|did == 88|did == 40| \
                             did == 139|did == 54|did == 59 ')

gw157a = df1_report1.query('did == 41|did == 42|did == 43|did == 55|did == 56')

gw157b = df1_report1.query('did == 60|did == 61|did == 62|did == 71|did == 72|did == 81|did == 82|did == 46|did == 47| did == 48| did == 49 \
                            |did == 63|did == 64|did == 65|did == 66|did == 73|did == 74|did == 75|did == 76|did == 83|did == 84|did == 85|did == 86')

entrance = df1_report1.query('did == 124|did == 125|did == 126|did == 127|did == 128|did == 129|did == 130|did == 131|did == 132| \
                              did == 133|did == 134|did == 135|did == 158  ')

gw154_gs = df1_report1.query('did == 142|did == 143|did == 146|did == 147')
gw155_gs = df1_report1.query('did == 144|did == 145|did == 148')
gw156a_gs = df1_report1.query('did == 151')
gw156c_gs = df1_report1.query('did == 152')
gw157b_gs = df1_report1.query('did == 149|did == 150|did == 153')

gw154 = gw154.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw155 = gw155.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw156a = gw156a.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw156b = gw156b.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw156c = gw156c.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw157a = gw157a.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw157b = gw157b.sort_values(by=['hist_dt'], axis = 0, ascending = True)
entrance = entrance.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw154_gs = gw154_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw155_gs = gw155_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw156a_gs = gw156a_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw156c_gs = gw156c_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw157b_gs = gw157b_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
gw154 = gw154.reset_index(drop=True)
gw155 = gw155.reset_index(drop=True)
gw156a = gw156a.reset_index(drop=True)
gw156b = gw156b.reset_index(drop=True)
gw156c = gw156c.reset_index(drop=True)
gw157a = gw157a.reset_index(drop=True)
gw157b = gw157b.reset_index(drop=True)
entrance = entrance.reset_index(drop=True)
gw154_gs = gw154_gs.reset_index(drop=True)
gw155_gs = gw155_gs.reset_index(drop=True)
gw156a_gs = gw156a_gs.reset_index(drop=True)
gw156c_gs = gw156c_gs.reset_index(drop=True)
gw157b_gs = gw157b_gs.reset_index(drop=True)
print('Task Completed')

Task Completed


In [93]:
did1 = gw154.query('did == 1')
did2 = gw154.query('did == 2')
did4 = gw154.query('did == 4')
did5 = gw154.query('did == 5')
did7 = gw154.query('did == 7')
did159 = gw154.query('did == 159')
did15 = gw154.query('did == 15')
did16 = gw154.query('did == 16')
did17 = gw154.query('did == 17')
did162 = gw154.query('did == 162')
did18 = gw154.query('did == 18')
did19 = gw154.query('did == 19')
did25 = gw154.query('did == 25')
did26 = gw154.query('did == 26')
did27 = gw154.query('did == 27')
did28 = gw154.query('did == 28')
did29 = gw154.query('did == 29')
did30 = gw154.query('did == 30')
did36 = gw154.query('did == 36')
did37 = gw154.query('did == 37')
did1 = did1.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did1 = did1.reset_index(drop=True)
did2 = did2.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did2 = did2.reset_index(drop=True)
did4 = did4.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did4 = did4.reset_index(drop=True)
did5 = did5.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did5 = did5.reset_index(drop=True)
did7 = did7.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did7 = did7.reset_index(drop=True)
did159 = did159.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did159 = did159.reset_index(drop=True)
did15 = did15.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did15 = did15.reset_index(drop=True)
did16 = did16.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did16 = did16.reset_index(drop=True)
did17 = did17.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did17 = did17.reset_index(drop=True)
did162 = did162.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did162 = did162.reset_index(drop=True)
did18 = did18.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did18 = did18.reset_index(drop=True)
did19 = did19.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did19 = did19.reset_index(drop=True)
did25 = did25.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did25 = did25.reset_index(drop=True)
did26 = did26.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did26 = did26.reset_index(drop=True)
did27 = did27.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did27 = did27.reset_index(drop=True)
did28 = did28.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did28 = did28.reset_index(drop=True)
did29 = did29.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did29 = did29.reset_index(drop=True)
did30 = did30.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did30 = did30.reset_index(drop=True)
did36 = did36.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did36 = did36.reset_index(drop=True)
did37 = did37.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did37 = did37.reset_index(drop=True)

did160 = gw155.query('did == 160')
did8 = gw155.query('did == 8')
did9 = gw155.query('did == 9')
did10 = gw155.query('did == 10')
did11 = gw155.query('did == 11')
did13 = gw155.query('did == 13')
did14 = gw155.query('did == 14')
did161 = gw155.query('did == 161')
did20 = gw155.query('did == 20')
did21 = gw155.query('did == 21')
did22 = gw155.query('did == 22')
did23 = gw155.query('did == 23')
did24 = gw155.query('did == 24')
did31 = gw155.query('did == 31')
did32 = gw155.query('did == 32')
did33 = gw155.query('did == 33')
did34 = gw155.query('did == 34')
did35 = gw155.query('did == 35')
did38 = gw155.query('did == 38')
did39 = gw155.query('did == 39')
did160 = did160.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did160 = did160.reset_index(drop=True)
did8 = did8.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did8 = did8.reset_index(drop=True)
did9 = did9.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did9 = did9.reset_index(drop=True)
did10 = did10.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did10 = did10.reset_index(drop=True)
did11 = did11.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did11 = did11.reset_index(drop=True)
did13 = did13.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did13 = did13.reset_index(drop=True)
did14 = did14.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did14 = did14.reset_index(drop=True)
did161 = did161.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did161 = did161.reset_index(drop=True)
did20 = did20.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did20 = did20.reset_index(drop=True)
did21 = did21.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did21 = did21.reset_index(drop=True)
did22 = did22.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did22 = did22.reset_index(drop=True)
did23 = did23.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did23 = did23.reset_index(drop=True)
did24 = did24.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did24 = did24.reset_index(drop=True)
did31 = did31.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did31 = did31.reset_index(drop=True)
did32 = did32.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did32 = did32.reset_index(drop=True)
did33 = did33.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did33 = did33.reset_index(drop=True)
did34 = did34.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did34 = did34.reset_index(drop=True)
did35 = did35.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did35 = did35.reset_index(drop=True)
did38 = did38.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did38 = did38.reset_index(drop=True)
did39 = did39.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did39 = did39.reset_index(drop=True)


did50 = gw156a.query('did == 50')
did51 = gw156a.query('did == 51')
did57 = gw156a.query('did == 57')
did58 = gw156a.query('did == 58')
did67 = gw156a.query('did == 67')
did68 = gw156a.query('did == 68')
did77 = gw156a.query('did == 77')
did78 = gw156a.query('did == 78')
did89 = gw156a.query('did == 89')
did90 = gw156a.query('did == 90')
did91 = gw156a.query('did == 91')
did92 = gw156a.query('did == 92')
did140 = gw156a.query('did == 140')
did141 = gw156a.query('did == 141')
did50 = did50.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did50 = did50.reset_index(drop=True)
did51 = did51.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did51 = did51.reset_index(drop=True)
did57 = did57.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did57 = did57.reset_index(drop=True)
did58 = did58.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did58 = did58.reset_index(drop=True)
did67 = did67.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did67 = did67.reset_index(drop=True)
did68 = did68.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did68 = did68.reset_index(drop=True)
did77 = did77.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did77 = did77.reset_index(drop=True)
did78 = did78.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did78 = did78.reset_index(drop=True)
did89 = did89.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did89 = did89.reset_index(drop=True)
did90 = did90.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did90 = did90.reset_index(drop=True)
did91 = did91.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did91 = did91.reset_index(drop=True)
did92 = did92.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did92 = did92.reset_index(drop=True)
did140 = did140.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did140 = did140.reset_index(drop=True)
did141 = did141.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did141 = did141.reset_index(drop=True)

did93 = gw156b.query('did == 93')
did93 = did93.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did93 = did93.reset_index(drop=True)

did52 = gw156c.query('did == 52')
did53 = gw156c.query('did == 53')
did69 = gw156c.query('did == 69')
did70 = gw156c.query('did == 70')
did79 = gw156c.query('did == 79')
did80 = gw156c.query('did == 80')
did87 = gw156c.query('did == 87')
did88 = gw156c.query('did == 88')
did40 = gw156c.query('did == 40')
did139 = gw156c.query('did == 139')
did54 = gw156c.query('did == 54')
did59 = gw156c.query('did == 59')
did52 = did52.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did52 = did52.reset_index(drop=True)
did53 = did53.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did53 = did53.reset_index(drop=True)
did69 = did69.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did69 = did69.reset_index(drop=True)
did70 = did70.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did70 = did70.reset_index(drop=True)
did79 = did79.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did79 = did79.reset_index(drop=True)
did80 = did80.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did80 = did80.reset_index(drop=True)
did87 = did87.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did87 = did87.reset_index(drop=True)
did88 = did88.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did88 = did88.reset_index(drop=True)
did40 = did40.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did40 = did40.reset_index(drop=True)
did139 = did139.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did139 = did139.reset_index(drop=True)
did54 = did54.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did54 = did54.reset_index(drop=True)
did59 = did59.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did59 = did59.reset_index(drop=True)

did41 = gw157a.query('did == 41')
did42 = gw157a.query('did == 42')
did43 = gw157a.query('did == 43')
did55 = gw157a.query('did == 55')
did56 = gw157a.query('did == 56')
did41 = did41.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did41 = did41.reset_index(drop=True)
did42 = did42.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did42 = did42.reset_index(drop=True)
did43 = did43.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did43 = did43.reset_index(drop=True)
did55 = did55.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did55 = did55.reset_index(drop=True)
did56 = did56.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did56 = did56.reset_index(drop=True)

did60 = gw157b.query('did == 60')
did61 = gw157b.query('did == 61')
did62 = gw157b.query('did == 62')
did71 = gw157b.query('did == 71')
did72 = gw157b.query('did == 72')
did81 = gw157b.query('did == 81')
did82 = gw157b.query('did == 82')
did46 = gw157b.query('did == 46')
did47 = gw157b.query('did == 47')
did48 = gw157b.query('did == 48')
did49 = gw157b.query('did == 49')
did63 = gw157b.query('did == 63')
did64 = gw157b.query('did == 64')
did65 = gw157b.query('did == 65')
did66 = gw157b.query('did == 66')
did73 = gw157b.query('did == 73')
did74 = gw157b.query('did == 74')
did75 = gw157b.query('did == 75')
did76 = gw157b.query('did == 76')
did83 = gw157b.query('did == 83')
did84 = gw157b.query('did == 84')
did85 = gw157b.query('did == 85')
did86 = gw157b.query('did == 86')
did60 = did60.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did60 = did60.reset_index(drop=True)
did61 = did61.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did61 = did61.reset_index(drop=True)
did62 = did62.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did62 = did62.reset_index(drop=True)
did71 = did71.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did71 = did71.reset_index(drop=True)
did72 = did72.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did72 = did72.reset_index(drop=True)
did81 = did81.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did81 = did81.reset_index(drop=True)
did82 = did82.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did82 = did82.reset_index(drop=True)
did46 = did46.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did46 = did46.reset_index(drop=True)
did47 = did47.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did47 = did47.reset_index(drop=True)
did48 = did48.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did48 = did48.reset_index(drop=True)
did49 = did49.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did49 = did49.reset_index(drop=True)
did63 = did63.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did63 = did63.reset_index(drop=True)
did64 = did64.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did64 = did64.reset_index(drop=True)
did65 = did65.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did65 = did65.reset_index(drop=True)
did66 = did66.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did66 = did66.reset_index(drop=True)
did73 = did73.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did73 = did73.reset_index(drop=True)
did74 = did74.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did74 = did74.reset_index(drop=True)
did75 = did75.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did75 = did75.reset_index(drop=True)
did76 = did76.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did76 = did76.reset_index(drop=True)
did83 = did83.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did83 = did83.reset_index(drop=True)
did84 = did84.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did84 = did84.reset_index(drop=True)
did85 = did85.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did85 = did85.reset_index(drop=True)
did86 = did86.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did86 = did86.reset_index(drop=True)

did124 = entrance.query('did == 124')
did125 = entrance.query('did == 125')
did126 = entrance.query('did == 126')
did127 = entrance.query('did == 127')
did128 = entrance.query('did == 128')
did129 = entrance.query('did == 129')
did130 = entrance.query('did == 130')
did131 = entrance.query('did == 131')
did132 = entrance.query('did == 132')
did133 = entrance.query('did == 133')
did134 = entrance.query('did == 134')
did135 = entrance.query('did == 135')
did158 = entrance.query('did == 158')
did124 = did124.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did124 = did124.reset_index(drop=True)
did125 = did125.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did125 = did125.reset_index(drop=True)
did126 = did126.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did126 = did126.reset_index(drop=True)
did127 = did127.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did127 = did127.reset_index(drop=True)
did128 = did128.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did128 = did128.reset_index(drop=True)
did129 = did129.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did129 = did129.reset_index(drop=True)
did130 = did130.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did130 = did130.reset_index(drop=True)
did131 = did131.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did131 = did131.reset_index(drop=True)
did132 = did132.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did132 = did132.reset_index(drop=True)
did133 = did133.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did133 = did133.reset_index(drop=True)
did134 = did134.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did134 = did134.reset_index(drop=True)
did135 = did135.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did135 = did135.reset_index(drop=True)
did158 = did158.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did158 = did158.reset_index(drop=True)

did142 = gw154_gs.query('did == 142')
did143 = gw154_gs.query('did == 143')
did146 = gw154_gs.query('did == 146')
did147 = gw154_gs.query('did == 147')
did142 = did142.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did142 = did142.reset_index(drop=True)
did143 = did143.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did143 = did143.reset_index(drop=True)
did146 = did146.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did146 = did146.reset_index(drop=True)
did147 = did147.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did147 = did147.reset_index(drop=True)

did144 = gw155_gs.query('did == 144')
did145 = gw155_gs.query('did == 145')
did148 = gw155_gs.query('did == 148')
did144 = did144.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did144 = did144.reset_index(drop=True)
did145 = did145.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did145 = did145.reset_index(drop=True)
did148 = did148.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did148 = did148.reset_index(drop=True)

did151 = gw156a_gs.query('did == 151')
did151 = did151.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did151 = did151.reset_index(drop=True)

did152 = gw156c_gs.query('did == 152')
did152 = did152.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did152 = did152.reset_index(drop=True)

did149 = gw157b_gs.query('did == 149')
did150 = gw157b_gs.query('did == 150')
did153 = gw157b_gs.query('did == 153')
did149 = did149.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did149 = did149.reset_index(drop=True)
did150 = did150.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did150 = did150.reset_index(drop=True)
did153 = did153.sort_values(by=['hist_dt'], axis = 0, ascending = True)
did153 = did153.reset_index(drop=True)
print('Task Completed')

Task Completed


In [2]:
df1_report1 = data1
df1_report1 = df1_report1.query('"2022-12-05 00:00:00"<= hist_dt <= "2022-12-11 23:59:59"')
#df1_report1 = df1_report1[df1_report1['report_uspace_yn'] == 'Y'] 
#df1_report1 = df1_report1.query('"2022-12-13 20:40:00"<= hist_dt <= "2022-12-31 23:59:59"')
lgw154 = df1_report1.query('did == 1|did == 2|did == 4|did == 5|did == 7|did == 159|did == 15|did == 16|did == 17|did == 162 \
                          |did == 18|did == 19|did == 25|did == 26|did == 27|did == 28|did == 29|did == 30|did == 36|did == 37')

lgw155 = df1_report1.query('did == 160|did == 8|did == 9|did == 10|did == 11|did == 13|did == 14|did == 161|did == 20|did == 21| \
                           did == 22|did == 23|did == 24|did == 31|did == 32|did == 33|did == 34|did == 35|did == 38|did == 39 ')

lgw156a = df1_report1.query('did == 50|did == 51|did == 57|did == 58|did == 67|did == 68|did == 77|did == 78|did == 89|did == 90| \
                            did == 91|did == 92|did == 140|did == 141 ')

lgw156b = df1_report1.query('did == 93')

lgw156c = df1_report1.query('did == 52|did == 53|did == 69|did == 70|did == 79|did == 80|did == 87|did == 88|did == 40| \
                             did == 139|did == 54|did == 59 ')

lgw157a = df1_report1.query('did == 41|did == 42|did == 43|did == 55|did == 56')

lgw157b = df1_report1.query('did == 60|did == 61|did == 62|did == 71|did == 72|did == 81|did == 82|did == 46|did == 47| did == 48| did == 49 \
                            |did == 63|did == 64|did == 65|did == 66|did == 73|did == 74|did == 75|did == 76|did == 83|did == 84|did == 85|did == 86')

lentrance = df1_report1.query('did == 124|did == 125|did == 126|did == 127|did == 128|did == 129|did == 130|did == 131|did == 132| \
                              did == 133|did == 134|did == 135|did == 158  ')

lgw154_gs = df1_report1.query('did == 142|did == 143|did == 146|did == 147')
lgw155_gs = df1_report1.query('did == 144|did == 145|did == 148')
lgw156a_gs = df1_report1.query('did == 151')
lgw156c_gs = df1_report1.query('did == 152')
lgw157b_gs = df1_report1.query('did == 149|did == 150|did == 153')

lgw154 = lgw154.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw155 = lgw155.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw156a = lgw156a.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw156b = lgw156b.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw156c = lgw156c.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw157a = lgw157a.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw157b = lgw157b.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lentrance = lentrance.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw154_gs = lgw154_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw155_gs = lgw155_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw156a_gs = lgw156a_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw156c_gs = lgw156c_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw157b_gs = lgw157b_gs.sort_values(by=['hist_dt'], axis = 0, ascending = True)
lgw154 = lgw154.reset_index(drop=True)
lgw155 = lgw155.reset_index(drop=True)
lgw156a = lgw156a.reset_index(drop=True)
lgw156b = lgw156b.reset_index(drop=True)
lgw156c = lgw156c.reset_index(drop=True)
lgw157a = lgw157a.reset_index(drop=True)
lgw157b = lgw157b.reset_index(drop=True)
lentrance = lentrance.reset_index(drop=True)
lgw154_gs = lgw154_gs.reset_index(drop=True)
lgw155_gs = lgw155_gs.reset_index(drop=True)
lgw156a_gs = lgw156a_gs.reset_index(drop=True)
lgw156c_gs = lgw156c_gs.reset_index(drop=True)
lgw157b_gs = lgw157b_gs.reset_index(drop=True)
print('Task Completed')

Task Completed


In [3]:
ldid1 = lgw154.query('did == 1')
ldid2 = lgw154.query('did == 2')
ldid4 = lgw154.query('did == 4')
ldid5 = lgw154.query('did == 5')
ldid7 = lgw154.query('did == 7')
ldid159 = lgw154.query('did == 159')
ldid15 = lgw154.query('did == 15')
ldid16 = lgw154.query('did == 16')
ldid17 = lgw154.query('did == 17')
ldid162 = lgw154.query('did == 162')
ldid18 = lgw154.query('did == 18')
ldid19 = lgw154.query('did == 19')
ldid25 = lgw154.query('did == 25')
ldid26 = lgw154.query('did == 26')
ldid27 = lgw154.query('did == 27')
ldid28 = lgw154.query('did == 28')
ldid29 = lgw154.query('did == 29')
ldid30 = lgw154.query('did == 30')
ldid36 = lgw154.query('did == 36')
ldid37 = lgw154.query('did == 37')
ldid1 = ldid1.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid1 = ldid1.reset_index(drop=True)
ldid2 = ldid2.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid2 = ldid2.reset_index(drop=True)
ldid4 = ldid4.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid4 = ldid4.reset_index(drop=True)
ldid5 = ldid5.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid5 = ldid5.reset_index(drop=True)
ldid7 = ldid7.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid7 = ldid7.reset_index(drop=True)
ldid159 = ldid159.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid159 = ldid159.reset_index(drop=True)
ldid15 = ldid15.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid15 = ldid15.reset_index(drop=True)
ldid16 = ldid16.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid16 = ldid16.reset_index(drop=True)
ldid17 = ldid17.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid17 = ldid17.reset_index(drop=True)
ldid162 = ldid162.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid162 = ldid162.reset_index(drop=True)
ldid18 = ldid18.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid18 = ldid18.reset_index(drop=True)
ldid19 = ldid19.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid19 = ldid19.reset_index(drop=True)
ldid25 = ldid25.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid25 = ldid25.reset_index(drop=True)
ldid26 = ldid26.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid26 = ldid26.reset_index(drop=True)
ldid27 = ldid27.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid27 = ldid27.reset_index(drop=True)
ldid28 = ldid28.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid28 = ldid28.reset_index(drop=True)
ldid29 = ldid29.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid29 = ldid29.reset_index(drop=True)
ldid30 = ldid30.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid30 = ldid30.reset_index(drop=True)
ldid36 = ldid36.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid36 = ldid36.reset_index(drop=True)
ldid37 = ldid37.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid37 = ldid37.reset_index(drop=True)


ldid160 = lgw155.query('did == 160')
ldid8 = lgw155.query('did == 8')
ldid9 = lgw155.query('did == 9')
ldid10 = lgw155.query('did == 10')
ldid11 = lgw155.query('did == 11')
ldid13 = lgw155.query('did == 13')
ldid14 = lgw155.query('did == 14')
ldid161 = lgw155.query('did == 161')
ldid20 = lgw155.query('did == 20')
ldid21 = lgw155.query('did == 21')
ldid22 = lgw155.query('did == 22')
ldid23 = lgw155.query('did == 23')
ldid24 = lgw155.query('did == 24')
ldid31 = lgw155.query('did == 31')
ldid32 = lgw155.query('did == 32')
ldid33 = lgw155.query('did == 33')
ldid34 = lgw155.query('did == 34')
ldid35 = lgw155.query('did == 35')
ldid38 = lgw155.query('did == 38')
ldid39 = lgw155.query('did == 39')
ldid160 = ldid160.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid160 = ldid160.reset_index(drop=True)
ldid8 = ldid8.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid8 = ldid8.reset_index(drop=True)
ldid9 = ldid9.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid9 = ldid9.reset_index(drop=True)
ldid10 = ldid10.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid10 = ldid10.reset_index(drop=True)
ldid11 = ldid11.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid11 = ldid11.reset_index(drop=True)
ldid13 = ldid13.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid13 = ldid13.reset_index(drop=True)
ldid14 = ldid14.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid14 = ldid14.reset_index(drop=True)
ldid161 = ldid161.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid161 = ldid161.reset_index(drop=True)
ldid20 = ldid20.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid20 = ldid20.reset_index(drop=True)
ldid21 = ldid21.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid21 = ldid21.reset_index(drop=True)
ldid22 = ldid22.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid22 = ldid22.reset_index(drop=True)
ldid23 = ldid23.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid23 = ldid23.reset_index(drop=True)
ldid24 = ldid24.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid24 = ldid24.reset_index(drop=True)
ldid31 = ldid31.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid31 = ldid31.reset_index(drop=True)
ldid32 = ldid32.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid32 = ldid32.reset_index(drop=True)
ldid33 = ldid33.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid33 = ldid33.reset_index(drop=True)
ldid34 = ldid34.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid34 = ldid34.reset_index(drop=True)
ldid35 = ldid35.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid35 = ldid35.reset_index(drop=True)
ldid38 = ldid38.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid38 = ldid38.reset_index(drop=True)
ldid39 = ldid39.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid39 = ldid39.reset_index(drop=True)


ldid50 = lgw156a.query('did == 50')
ldid51 = lgw156a.query('did == 51')
ldid57 = lgw156a.query('did == 57')
ldid58 = lgw156a.query('did == 58')
ldid67 = lgw156a.query('did == 67')
ldid68 = lgw156a.query('did == 68')
ldid77 = lgw156a.query('did == 77')
ldid78 = lgw156a.query('did == 78')
ldid89 = lgw156a.query('did == 89')
ldid90 = lgw156a.query('did == 90')
ldid91 = lgw156a.query('did == 91')
ldid92 = lgw156a.query('did == 92')
ldid140 = lgw156a.query('did == 140')
ldid141 = lgw156a.query('did == 141')
ldid50 = ldid50.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid50 = ldid50.reset_index(drop=True)
ldid51 = ldid51.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid51 = ldid51.reset_index(drop=True)
ldid57 = ldid57.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid57 = ldid57.reset_index(drop=True)
ldid58 = ldid58.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid58 = ldid58.reset_index(drop=True)
ldid67 = ldid67.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid67 = ldid67.reset_index(drop=True)
ldid68 = ldid68.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid68 = ldid68.reset_index(drop=True)
ldid77 = ldid77.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid77 = ldid77.reset_index(drop=True)
ldid78 = ldid78.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid78 = ldid78.reset_index(drop=True)
ldid89 = ldid89.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid89 = ldid89.reset_index(drop=True)
ldid90 = ldid90.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid90 = ldid90.reset_index(drop=True)
ldid91 = ldid91.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid91 = ldid91.reset_index(drop=True)
ldid92 = ldid92.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid92 = ldid92.reset_index(drop=True)
ldid140 = ldid140.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid140 = ldid140.reset_index(drop=True)
ldid141 = ldid141.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid141 = ldid141.reset_index(drop=True)



ldid93 = lgw156b.query('did == 93')
ldid93 = ldid93.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid93 = ldid93.reset_index(drop=True)



ldid52 = lgw156c.query('did == 52')
ldid53 = lgw156c.query('did == 53')
ldid69 = lgw156c.query('did == 69')
ldid70 = lgw156c.query('did == 70')
ldid79 = lgw156c.query('did == 79')
ldid80 = lgw156c.query('did == 80')
ldid87 = lgw156c.query('did == 87')
ldid88 = lgw156c.query('did == 88')
ldid40 = lgw156c.query('did == 40')
ldid139 = lgw156c.query('did == 139')
ldid54 = lgw156c.query('did == 54')
ldid59 = lgw156c.query('did == 59')
ldid52 = ldid52.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid52 = ldid52.reset_index(drop=True)
ldid53 = ldid53.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid53 = ldid53.reset_index(drop=True)
ldid69 = ldid69.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid69 = ldid69.reset_index(drop=True)
ldid70 = ldid70.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid70 = ldid70.reset_index(drop=True)
ldid79 = ldid79.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid79 = ldid79.reset_index(drop=True)
ldid80 = ldid80.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid80 = ldid80.reset_index(drop=True)
ldid87 = ldid87.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid87 = ldid87.reset_index(drop=True)
ldid88 = ldid88.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid88 = ldid88.reset_index(drop=True)
ldid40 = ldid40.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid40 = ldid40.reset_index(drop=True)
ldid139 = ldid139.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid139 = ldid139.reset_index(drop=True)
ldid54 = ldid54.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid54 = ldid54.reset_index(drop=True)
ldid59 = ldid59.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid59 = ldid59.reset_index(drop=True)



ldid41 = lgw157a.query('did == 41')
ldid42 = lgw157a.query('did == 42')
ldid43 = lgw157a.query('did == 43')
ldid55 = lgw157a.query('did == 55')
ldid56 = lgw157a.query('did == 56')
ldid41 = ldid41.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid41 = ldid41.reset_index(drop=True)
ldid42 = ldid42.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid42 = ldid42.reset_index(drop=True)
ldid43 = ldid43.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid43 = ldid43.reset_index(drop=True)
ldid55 = ldid55.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid55 = ldid55.reset_index(drop=True)
ldid56 = ldid56.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid56 = ldid56.reset_index(drop=True)



ldid60 = lgw157b.query('did == 60')
ldid61 = lgw157b.query('did == 61')
ldid62 = lgw157b.query('did == 62')
ldid71 = lgw157b.query('did == 71')
ldid72 = lgw157b.query('did == 72')
ldid81 = lgw157b.query('did == 81')
ldid82 = lgw157b.query('did == 82')
ldid46 = lgw157b.query('did == 46')
ldid47 = lgw157b.query('did == 47')
ldid48 = lgw157b.query('did == 48')
ldid49 = lgw157b.query('did == 49')
ldid63 = lgw157b.query('did == 63')
ldid64 = lgw157b.query('did == 64')
ldid65 = lgw157b.query('did == 65')
ldid66 = lgw157b.query('did == 66')
ldid73 = lgw157b.query('did == 73')
ldid74 = lgw157b.query('did == 74')
ldid75 = lgw157b.query('did == 75')
ldid76 = lgw157b.query('did == 76')
ldid83 = lgw157b.query('did == 83')
ldid84 = lgw157b.query('did == 84')
ldid85 = lgw157b.query('did == 85')
ldid86 = lgw157b.query('did == 86')
ldid60 = ldid60.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid60 = ldid60.reset_index(drop=True)
ldid61 = ldid61.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid61 = ldid61.reset_index(drop=True)
ldid62 = ldid62.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid62 = ldid62.reset_index(drop=True)
ldid71 = ldid71.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid71 = ldid71.reset_index(drop=True)
ldid72 = ldid72.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid72 = ldid72.reset_index(drop=True)
ldid81 = ldid81.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid81 = ldid81.reset_index(drop=True)
ldid82 = ldid82.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid82 = ldid82.reset_index(drop=True)
ldid46 = ldid46.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid46 = ldid46.reset_index(drop=True)
ldid47 = ldid47.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid47 = ldid47.reset_index(drop=True)
ldid48 = ldid48.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid48 = ldid48.reset_index(drop=True)
ldid49 = ldid49.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid49 = ldid49.reset_index(drop=True)
ldid63 = ldid63.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid63 = ldid63.reset_index(drop=True)
ldid64 = ldid64.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid64 = ldid64.reset_index(drop=True)
ldid65 = ldid65.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid65 = ldid65.reset_index(drop=True)
ldid66 = ldid66.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid66 = ldid66.reset_index(drop=True)
ldid73 = ldid73.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid73 = ldid73.reset_index(drop=True)
ldid74 = ldid74.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid74 = ldid74.reset_index(drop=True)
ldid75 = ldid75.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid75 = ldid75.reset_index(drop=True)
ldid76 = ldid76.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid76 = ldid76.reset_index(drop=True)
ldid83 = ldid83.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid83 = ldid83.reset_index(drop=True)
ldid84 = ldid84.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid84 = ldid84.reset_index(drop=True)
ldid85 = ldid85.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid85 = ldid85.reset_index(drop=True)
ldid86 = ldid86.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid86 = ldid86.reset_index(drop=True)



ldid124 = lentrance.query('did == 124')
ldid125 = lentrance.query('did == 125')
ldid126 = lentrance.query('did == 126')
ldid127 = lentrance.query('did == 127')
ldid128 = lentrance.query('did == 128')
ldid129 = lentrance.query('did == 129')
ldid130 = lentrance.query('did == 130')
ldid131 = lentrance.query('did == 131')
ldid132 = lentrance.query('did == 132')
ldid133 = lentrance.query('did == 133')
ldid134 = lentrance.query('did == 134')
ldid135 = lentrance.query('did == 135')
ldid158 = lentrance.query('did == 158')
ldid124 = ldid124.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid124 = ldid124.reset_index(drop=True)
ldid125 = ldid125.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid125 = ldid125.reset_index(drop=True)
ldid126 = ldid126.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid126 = ldid126.reset_index(drop=True)
ldid127 = ldid127.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid127 = ldid127.reset_index(drop=True)
ldid128 = ldid128.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid128 = ldid128.reset_index(drop=True)
ldid129 = ldid129.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid129 = ldid129.reset_index(drop=True)
ldid130 = ldid130.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid130 = ldid130.reset_index(drop=True)
ldid131 = ldid131.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid131 = ldid131.reset_index(drop=True)
ldid132 = ldid132.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid132 = ldid132.reset_index(drop=True)
ldid133 = ldid133.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid133 = ldid133.reset_index(drop=True)
ldid134 = ldid134.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid134 = ldid134.reset_index(drop=True)
ldid135 = ldid135.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid135 = ldid135.reset_index(drop=True)
ldid158 = ldid158.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid158 = ldid158.reset_index(drop=True)



ldid142 = lgw154_gs.query('did == 142')
ldid143 = lgw154_gs.query('did == 143')
ldid146 = lgw154_gs.query('did == 146')
ldid147 = lgw154_gs.query('did == 147')
ldid142 = ldid142.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid142 = ldid142.reset_index(drop=True)
ldid143 = ldid143.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid143 = ldid143.reset_index(drop=True)
ldid146 = ldid146.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid146 = ldid146.reset_index(drop=True)
ldid147 = ldid147.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid147 = ldid147.reset_index(drop=True)



ldid144 = lgw155_gs.query('did == 144')
ldid145 = lgw155_gs.query('did == 145')
ldid148 = lgw155_gs.query('did == 148')
ldid144 = ldid144.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid144 = ldid144.reset_index(drop=True)
ldid145 = ldid145.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid145 = ldid145.reset_index(drop=True)
ldid148 = ldid148.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid148 = ldid148.reset_index(drop=True)



ldid151 = lgw156a_gs.query('did == 151')
ldid151 = ldid151.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid151 = ldid151.reset_index(drop=True)

ldid152 = lgw156c_gs.query('did == 152')
ldid152 = ldid152.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid152 = ldid152.reset_index(drop=True)

ldid149 = lgw157b_gs.query('did == 149')
ldid150 = lgw157b_gs.query('did == 150')
ldid153 = lgw157b_gs.query('did == 153')
ldid149 = ldid149.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid149 = ldid149.reset_index(drop=True)
ldid150 = ldid150.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid150 = ldid150.reset_index(drop=True)
ldid153 = ldid153.sort_values(by=['hist_dt'], axis = 0, ascending = True)
ldid153 = ldid153.reset_index(drop=True)

print('Task Completed')

Task Completed


In [76]:
LIDX = did141
LLID= ldid141
i = 0
j = 0
k = 0
#LID1 = LIDX[LIDX.mid == 0]
LIDX['hist_dt'] = pd.to_datetime(LIDX['hist_dt'])
LID = LIDX.reset_index(drop=True)
#length = len(did160_complete)
complete_list2 = []
for abyscuss in range (len(LID)):
    alpha = []
    for bravo in range(4):
        alpha.append(0)
    complete_list2.append(alpha)
    

    
for i in range(len(LID)):
    complete_list2[i][0] = LLID.loc[i ,'hist_dt']
    complete_list2[i][1] = LID.loc[i ,'device_fault']
    complete_list2[i][2] = LLID.loc[i ,'onoff']
    complete_list2[i][3] = LLID.loc[i ,'level']
    
complete_list3 = pd.DataFrame(complete_list2, columns=['hist_dt','device_fault','onoff','level'])
complete_list3

i = 0
j = 0
k = 0

complete_list5 = []
for viscosity in range (len(LID)):
    echo = []
    for kilo in range(2):
        echo.append(0)
    complete_list5.append(echo)
    
    
for i in range(len(LID)):
    complete_list5[i][0] = complete_list2[i][0]
    if complete_list2[i][1] == 1:
        complete_list5[i][1] = -20
    if complete_list2[i][1] == 0:
        if complete_list2[i][2] == 0:
            complete_list5[i][1] = 0
        if complete_list2[i][2] == 1:
            complete_list5[i][1] = complete_list2[i][3]
            
complete_list6 = pd.DataFrame(complete_list5, columns=['hist_dt','level'])
complete_list6
complete_list7 = [every_each_min1 for n1,every_each_min1 in complete_list6.set_index('hist_dt').groupby(pd.Grouper(freq='D'))]            


day19 = complete_list7[0]
day19 = day19.reset_index(drop=False)
day20 = complete_list7[1]
day20 = day20.reset_index(drop=False)
day21 = complete_list7[2]
day21 = day21.reset_index(drop=False)
day22 = complete_list7[3]
day22 = day22.reset_index(drop=False)
day23 = complete_list7[4]
day23 = day23.reset_index(drop=False)
day24 = complete_list7[5]
day24 = day24.reset_index(drop=False)
day25 = complete_list7[6]
day25 = day25.reset_index(drop=False)
print('task completed')

task completed


In [9]:
complete_list6 = ldid50
complete_list7 = [every_each_min1 for n1,every_each_min1 in complete_list6.set_index('hist_dt').groupby(pd.Grouper(freq='D'))]            


day5 = complete_list7[0]
day5 = day5.reset_index(drop=False)
day6 = complete_list7[1]
day6 = day6.reset_index(drop=False)
day7 = complete_list7[2]
day7 = day7.reset_index(drop=False)
day8 = complete_list7[3]
day8 = day8.reset_index(drop=False)
day9 = complete_list7[4]
day9 = day9.reset_index(drop=False)
day10 = complete_list7[5]
day10 = day10.reset_index(drop=False)
day11 = complete_list7[6]
day11 = day11.reset_index(drop=False)

In [10]:
import plotly.express as px
import plotly.io as po
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig1 = px.scatter(day5,x='hist_dt',y='level',color='level',template='ggplot2',title='did 50  (DEC/05/22)')
#fig2 = px.scatter(day20,x='hist_dt',y='level',color='level',template='ggplot2',title='LID 50 (DEC/20/22)')
#fig3 = px.scatter(day21,x='hist_dt',y='level',color='level',template='ggplot2',title='LID 50 (DEC/21/22)')
#fig4 = px.scatter(day22,x='hist_dt',y='level',color='level',template='ggplot2',title='LID 38 (DEC/22/22)')
#fig5 = px.scatter(day23,x='hist_dt',y='level',color='level',template='ggplot2',title='LID 38 (DEC/23/22)')
#fig6 = px.scatter(day24,x='hist_dt',y='level',color='level',template='ggplot2',title='LID 38 (DEC/24/22)')
#fig7 = px.scatter(day25,x='hist_dt',y='level',color='level',template='ggplot2',title='LID 38 (DEC/25/22)')
#fig7 = px.scatter(day13,x='hist_dt',y='level',width=800,height=200)
#fig1.add_trace(go.scatter(marker_colorscale='Viridis'))
fig1.update_layout(coloraxis_showscale=False)
#fig2.update_layout(coloraxis_showscale=False)
#fig3.update_layout(coloraxis_showscale=False)
#fig4.update_layout(coloraxis_showscale=False)
#fig5.update_layout(coloraxis_showscale=False)
#fig6.update_layout(coloraxis_showscale=False)
#fig7.update_layout(coloraxis_showscale=False)


fig1.show()
#fig2.show()
#fig3.show()
#fig4.show()
#fig5.show()
#fig6.show()
#fig7.show()
fig1.write_html('did50_day05.html')
#fig2.write_html('LID38_day20.html')
#fig3.write_html('LID38_day21.html')
#fig4.write_html('LID38_day22.html')
#fig5.write_html('LID38_day23.html')
#fig6.write_html('LID38_day24.html')
#fig7.write_html('LID38_day25.html')